In [ ]:
%load_ext autoreload
%autoreload 1

%aimport Seasonal_Outliers
import importlib

importlib.reload(Seasonal_Outliers)

import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy import stats
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable

import Ipynb_importer
from numba import jit,cuda


from prophet import Prophet
import logging
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

def read(file):
    df = pd.read_csv(file)
    print(df.head())
    
    
    df['started_datetime'] = pd.to_datetime(df['started_datetime'], errors='coerce')
    df['started_date'] = df['started_datetime'].dt.date
    
    df['ended_datetime'] = pd.to_datetime(df['ended_datetime'], errors='coerce')
    df['ended_date'] = df['ended_datetime'].dt.date

    
    df['started_datetime'] = df['started_datetime'].dt.round(freq='1h').dt.ceil(freq='2h').dt.floor(freq='4h')
    df['ended_datetime'] = df['ended_datetime'].dt.round(freq='1h').dt.ceil(freq='2h').dt.floor(freq='4h')
    
    df.drop(['start_station_id', 'end_station_id'], axis=1,inplace=True)
    
    df['Day of week'] = df['started_datetime'].dt.dayofweek
    dt_wkends = df[df['Day of week']>4]['started_date'].unique()
    
    return df,dt_wkends.tolist()


def combine_df(idir,files):
    df = pd.DataFrame()
    dt_non_work = []#non-working day
    for fi in files:
        dfi,dt_wkends = read(idir+fi)
        df = df.append(dfi,ignore_index=True)
        dt_non_work+=dt_wkends
    holiday = pd.to_datetime(pd.Series(['2021-07-04','2021-07-05','2021-09-06','2021-09-16','2021-10-11'])).dt.date.unique()
    dt_non_work = np.append(dt_non_work,holiday)#Labor Day

    return df,dt_non_work


def vis(idir,outliers,col,title):
    
    start = pd.to_datetime('07/01/2021 00:00:00')
    end = pd.to_datetime('10/31/2021 23:59:59')
    dt_range = pd.to_datetime(pd.date_range(start=start, end=end,freq='4H'),format='%Y-%m-%d %H:%M:%S')
    df = pd.DataFrame(index=outliers[col[0]].unique(),columns=dt_range)
    df = df.fillna(0).astype('float')
    for idx in df.index:
        vals = outliers[outliers[col[0]]==idx][[col[1]]].values
        for val in vals:
            if val in dt_range.values:
                df.loc[idx][val] = outliers.loc[(outliers[col[0]]==idx)&(outliers[col[1]]==val[0])]['norm_resid']#.values[0]

    figsize=(70, 30)
    fig, ax = plt.subplots(figsize=figsize)
    # my_colors=['whitesmoke','orangered']
    
    ax = sns.heatmap(df, cmap='bwr', center=0, xticklabels = df.columns.strftime('%Y-%m-%d,%H'),
                     cbar_kws={"shrink":0.5},zorder=1)#square=True,
    
    for i in range(df.shape[0]+1):
        ax.axhline(i, color='white', lw=0.5,zorder=2)
    y1 = 0
    y2 = len(df)

    # ida_window = pd.to_datetime(['2021-08-26 00','2021-09-04 00'])
    # ida_nyc_window = pd.to_datetime(['2021-09-01 00','2021-09-02 00'])
    xlabels = ax.get_xticklabels()
    print(xlabels[0].get_text())
    print(xlabels[0])

    xs = [ x.get_position()[0] for x in xlabels if str(x.get_text()) in ["2021-09-01,00","2021-09-03,00"]]
    x1 = xs[0]-0.5
    x2 = xs[1]+0.5
    print(xs)
    # ax.fill_between([x1,x2],ida_window[0],ida_window[1], color='lightskyblue', alpha=0.4)
    ax.fill_between([x1,x2],y1,y2,alpha=0.5, color='none',edgecolor='blue', zorder=3, label='Ida',linewidth=3)
    
    xs2 = [ x.get_position()[0] for x in xlabels if str(x.get_text()) in ["2021-08-21,00","2021-08-24,00"]]
    x12 = xs2[0]-0.5
    x22 = xs2[1]+0.5
    print(xs2)
    ax.fill_between([x12,x22],y1,y2,alpha=0.8, color='none',edgecolor='gold', zorder=3,label='Henri',linewidth=3)
    
    xs3 = [ x.get_position()[0] for x in xlabels if str(x.get_text()) in ["2021-07-08,00","2021-07-10,00"]]
    x13 = xs3[0]-0.5
    x23 = xs3[1]+0.5
    print(xs3)
    ax.fill_between([x13,x23],y1,y2, alpha=0.5, color='none',edgecolor='green', zorder=3, label='Elsa',linewidth=3)
    
    xs4 = [ x.get_position()[0] for x in xlabels if str(x.get_text()) in ["2021-10-25,00","2021-10-27,00"]]
    x14 = xs4[0]-0.5
    x24 = xs4[1]+0.5
    print(xs4)
    ax.fill_between([x14,x24],y1,y2, alpha=0.5, color='none',edgecolor='deeppink', zorder=3,label="Nor'eastern",linewidth=3)
    
    '''
    xticks = ax.get_xticks()
    for i, tick in enumerate(xticks):
        if i % 2 == 0:
            ax.xaxis.get_major_ticks()[i].set_visible(True)
        else:
            ax.xaxis.get_major_ticks()[i].set_visible(False)
    '''
    for i, label in enumerate(xlabels):
        if i % 2 == 0:
            label.set_visible(True)
        else:
            label.set_visible(False)
    if "Line's" in title:
        ax.tick_params(axis='x', labelsize=7)    
    # ax.set_title(title)
    ax.legend(ncol=4, loc='upper right',fontsize=40)

    fig = plt.gcf()
    cax = fig.axes[-1]
    plt.subplots_adjust(left=0.15, right=0.95, bottom=0.15, top=0.9, wspace=0.2, hspace=0.2)
    cax.set_position([.796, .2, .03, .6]) 
    plt.savefig(idir+'result/'+title.replace(':','_').replace('/',' ')+'.png',format='png',bbox_inches='tight',dpi=300)
    plt.show()


    
@jit(target_backend='cuda')      
def find_outliers(idir,df,title):
    if "starts" in title:
        col = ["start_station_name",'started_datetime']
    elif "ends" in title:
        col = ["end_station_name",'ended_datetime']
    gps = df.groupby([col[0]])
    # names = df[col[0]].unique()
    outs = pd.DataFrame()
    stas = pd.DataFrame()
    for nm,df2 in gps:
        # if len(df2)<200:
        #     continue
        # print(df2)
        nm = str(nm)
        # print(nm)
        df2['index'] = df2.index
        sta_df = pd.DataFrame()
        sta_df['cnt'] = df2.groupby([col[1]]).count()['index']
        key='cnt'
        if (sta_df['cnt'].mean()<6):
            continue
        
        out = pd.DataFrame()
        tmp,forecast = Seasonal_Outliers.seasonal_de_hour(sta_df,key,plot=False)
        if len(tmp)>0:
            out[col[1]] = tmp
            out[col[0]] = nm
            outs = pd.concat([outs,out[col]],ignore_index=True)
            forecast[col] = nm
            stas = pd.concat([stas,forecast],ignore_index=True)
    
    sst = stas[['ds','norm_resid',col[0]]]
    print(sst,outs)
    sst['ds'] = pd.to_datetime(sst['ds'], errors='coerce')
    sst[col[1]] = sst['ds']
    outs = pd.merge(outs,sst,on=col)
    print(outs)
    max_value = outs['norm_resid'].max()
    min_value = outs['norm_resid'].min()
    
    vis(idir,outs,col, title)
    stas.to_csv(idir+'result/'+title.replace(':','_').replace('/',' ')+'.csv',index=False)


    
    
def run(idir,files,_title):
        
    df,dt_non_work = combine_df(idir,files)
    


    keys_dict = {
        "stations' starts":[''],
        "stations' ends":[''],
                 
                }
    
    for k,v in keys_dict.items():
        if "starts" in k:
            title = _title + "stations' starts"
        if "ends" in k:
            title = _title + "stations' ends"
        find_outliers(idir,df,title)
        
 


    


if __name__=='__main__':
    
    input_dir = ""
    files = [i for i in os.listdir(input_dir) if os.path.splitext(i)[1] == '.csv']
    
    _title = 'Hourly Daily Stationarity of '
    
    run(input_dir, files, _title)


